In [1]:
import matplotlib.pyplot as plt
import numpy as np
import glob
from PIL import Image
import tensorflow as tf

import stylegan_two
from stylegan_two import StyleGAN

In [2]:
stylegan_two.im_size = 128
stylegan_two.directory = "frames/"

class OptimisationEncoder:
    def __init__(self, model, batch_size=16) :
        self.model = model
        self.batch_size = batch_size
        self.steps = [(0.1, 200),(0.05, 100),(0.01, 100),(0.005,100)]
        self.loss_hist = []
        self.batch_size = batch_size
        self.n1 = stylegan_two.noiseList(1)
    
    def _opt_step(self, x1,x2, trunc,target,lr):
        with tf.GradientTape(persistent=True) as g:
            result = self.model.GAN.GMA(stylegan_two.n_layers*[x1] + [x2,trunc])
            loss = tf.reduce_sum(tf.abs(result-target))
        dx2 = g.gradient(loss, x2)
        #x1.assign(x1 - lr * dx1)
        x2.assign(x2 - lr * dx2)
        return loss
    
    def _encode_batch(self, paths) :
        n = len(paths)
        n1 = np.repeat(self.n1[0], n, axis=0)
        x1 = tf.Variable(n1)
        n2 = np.random.uniform(0.0, 1.0, size = [n, stylegan_two.im_size, stylegan_two.im_size, 1]).astype('float32')
        x2 = tf.Variable(n2)
        trunc = tf.ones([1, 1]) * 1.0
        target = []
        for path in paths :
            im = Image.open(path)
            im = im.resize((stylegan_two.im_size, stylegan_two.im_size))
            target.append(im)
        target = tf.constant(np.array(im).astype(np.float32)/255)    
        
        hist = []
        for lr,nb in self.steps :
            for i in range(nb) :
                l = self._opt_step(x1,x2,trunc,target,lr)
                hist.append(l.numpy())
        self.loss_hist.append(hist)
        
        return x2.numpy()
    
    def encode_images(self, paths) :
        i = 0
        res = []
        while i < len(paths) :
            next_i = min(len(paths), i+self.batch_size)
            res.append(self._encode_batch(paths[i:next_i]))
            i = next_i
            print(next_i/len(paths))
        return np.concatenate(res)
    
    def transform(self, lattents) :
        images = []
        trunc = np.ones([1, 1]) * 1.0
        for lattent in lattents :
            n2 = tf.expand_dims(lattent, 0)
            images.append(self.model.GAN.GMA(self.n1 + [n2])[0].numpy())
        return images

In [ ]:
model = StyleGAN(lr = 0.0001, silent = False)
model.load(10)

In [ ]:
paths = glob.glob('frames/*.jpg')
encoder = OptimisationEncoder(model)

In [ ]:
res = encoder.encode_images(paths)

In [ ]:
images = encoder.transform(res)

In [ ]:
plt.plot(encoder.loss_hist[0])

In [ ]:
plt.imshow(images[1])

In [ ]:
save_paths = [s.replace("frames", "decoded_frames") for s in paths]

In [ ]:
for path, image in zip (save_paths, images) :
    im = Image.fromarray((np.clip(images[0], 0, 1)*255).astype(np.uint8))
    im.save(path)